In [ ]:
# 일단 보내주신 api 호출 건을 바탕으로 다시 코드를 작성하긴 했습니다.
# 데이터로드 -> 강화학습 환경 정의 -> 강화학습 모델 정의 -> 신세계몰 api 연동 -> 추천 시스템 및 강화학습 모델 통합 -> 성능평가 순으로 진행되야 할 것 같습니다.
# 이미지 데이터는 amazon s3을 통해 url을 추출할 예정입니다.
# 제가 api 호출 건까지 합쳐 성능평가를 진행할 수 없기에, product table에 임시 적재하여 성능평가를 먼저 완료할 계획입니다.
# 실제 api 호출 시, product table은 필요 없습니다.
# AI 모듈의 f1-score, 누적 reward로 평가할 예정입니다.
# 제가 일요일까지 이 코드를 임시 평가 마친 후, 하이퍼 파라미터를 조정한 후, 데이터를 json파일로 저장하고, AI 모듈을 json파일로 저장하겠습니다.
# 제 역량은 아마 여기까지 일 것 같고, 추후 유빈님께서 진행하면 될 것 같습니다!

In [ ]:
# 혹시 몰라, 벡엔드에서 json파일 불러오는 코드를 작성했습니다.
import json
import pandas as pd

# JSON 파일 로드
food_data = pd.read_json('food_data.json')
recipe_data = pd.read_json('recipe_data.json')
user_data = pd.read_json('user_data.json')
# 만약, 제가 추후에 보내드릴 json파일이 손상됐을 경우, 이런 식으로 불러오시면 됩니다.

print(food_data.head())
print(recipe_data.head())
print(user_data.head())

from tensorflow.keras.models import model_from_json

# 모델 구조 로드
with open("model.json", "r") as json_file:
    loaded_model_json = json_file.read()
loaded_model = model_from_json(loaded_model_json)

# 모델 가중치 로드
loaded_model.load_weights("model.h5")

# 모델 컴파일
loaded_model.compile(loss='mse', optimizer='adam')

# 모델 사용 예시
import numpy as np

# 예시 입력 데이터
input_data = np.random.rand(1, 5)

# 예측 수행
predictions = loaded_model.predict(input_data)
print(predictions)

In [ ]:
# 여기부턴 AI 모듈 구현 코드입니다.
# 혹시 몰라 설명을 다 적어놓겠습니다.

In [ ]:
import pandas as pd

# 데이터 로드
allergy_table = pd.read_csv('allergy_table.csv')
food_table = pd.read_csv('food_table.csv')
fridge_contents_table = pd.read_csv('fridge_contents_table.csv')
recipe_ingredients_table = pd.read_csv('recipe_ingredients_table.csv')
recipes_table = pd.read_csv('recipes_table.csv')
shopping_lists_table = pd.read_csv('shopping_lists_table.csv')
user_allergies_table = pd.read_csv('user_allergies_table.csv')
user_data_table = pd.read_csv('user_data_table.csv')
user_preferences_table = pd.read_csv('user_preferences_table.csv')

In [ ]:
import numpy as np

class FridgeEnv:
    def __init__(self, fridge_contents_table, food_table, user_data_table, user_id):
        self.fridge_contents = fridge_contents_table[fridge_contents_table['user_id'] == user_id]
        self.food_table = food_table
        self.user_data = user_data_table[user_data_table['user_id'] == user_id]
        self.state_size = len(self.fridge_contents)
        self.action_size = len(food_table)
        self.reset()

    def reset(self):
        self.state = self.fridge_contents['quantity'].values
        return self.state

    def step(self, action):
        food_id = self.food_table.iloc[action]['food_id']
        food_quantity = 1  # 기본 주문 수량

        if food_id in self.fridge_contents['food_id'].values:
            idx = self.fridge_contents[self.fridge_contents['food_id'] == food_id].index[0]
            self.fridge_contents.at[idx, 'quantity'] += food_quantity
        else:
            new_entry = {'user_id': self.user_data['user_id'].values[0], 'food_id': food_id, 'quantity': food_quantity}
            self.fridge_contents = self.fridge_contents.append(new_entry, ignore_index=True)

        reward = 1 if food_quantity > 0 else 0
        done = False
        next_state = self.fridge_contents['quantity'].values

        return next_state, reward, done, {}

    def get_state(self):
        return self.state

# 사용자 ID 설정
user_id = 1

# 환경 초기화
env = FridgeEnv(fridge_contents_table, food_table, user_data_table, user_id)

In [ ]:
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

class DQNAgent:
    def __init__(self, state_size, action_size, access_token):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.access_token = access_token

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

    def order_food(self, product_id, quantity):
        # 신세계몰 API 호출
        return add_to_cart(self.access_token, product_id, quantity)

In [ ]:
import requests

# API 기본 URL
BASE_URL = "https://api.shinsegaemall.com"

# API 호출 함수
def call_api(endpoint, method='GET', headers=None, data=None, params=None):
    url = f"{BASE_URL}{endpoint}"
    response = requests.request(method, url, headers=headers, json=data, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

# 로그인 함수
def login(email, password):
    endpoint = "/members/login"
    data = {
        "email": email,
        "password": password
    }
    response = call_api(endpoint, method='POST', data=data)
    return response['code'], response['data']['accessToken']

# 장바구니 조회 함수
def get_cart(access_token):
    endpoint = "/products/list"
    headers = {
        "accessToken": access_token
    }
    return call_api(endpoint, headers=headers)

# 장바구니에 품목 추가 함수
def add_to_cart(access_token, product_id, quantity):
    endpoint = "/products/count"
    headers = {
        "accessToken": access_token
    }
    data = {
        "productId": product_id,
        "quantity": quantity
    }
    return call_api(endpoint, method='PATCH', headers=headers, data=data)

In [ ]:
def recommend_recipes(user_id, fridge_contents, recipes_table, recipe_ingredients_table):
    recommended_recipes = []
    for recipe_id in recipes_table['recipe_id'].unique():
        ingredients = recipe_ingredients_table[recipe_ingredients_table['recipe_id'] == recipe_id]
        if all(item in fridge_contents['food_id'].values for item in ingredients['food_id'].values):
            recommended_recipes.append(recipe_id)
    return recommended_recipes

def evaluate_recommender(agent, env, users, recipes, interactions, num_recipes=5):
    test_users = interactions['user_id'].unique()
    precisions, recalls, f1_scores = [], [], []

    for user_id in test_users:
        env.reset()
        state = env.get_state()
        recommended_recipes = agent.act(state)

        actual_items = set(interactions[interactions['user_id'] == user_id]['recipe_id'])
        recommended_items = set(recommend_recipes(user_id, env.fridge_contents, recipes, interactions))

        tp = len(actual_items & recommended_items)
        fp = len(recommended_items - actual_items)
        fn = len(actual_items - recommended_items)

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    average_precision = sum(precisions) / len(precisions)
    average_recall = sum(recalls) / len(recalls)
    average_f1_score = sum(f1_scores) / len(f1_scores)

    print(f'Average Precision: {average_precision}')
    print(f'Average Recall: {average_recall}')
    print(f'Average F1-Score: {average_f1_score}')

    return average_f1_score

# 사용자 데이터 로드
users = user_data_table
recipes = recipes_table
interactions = user_preferences_table

# 강화학습 모델 학습 및 평가
state_size = env.state_size
action_size = env.action_size
email = "your_email@example.com"
password = "your_password"
status_code, access_token = login(email, password)
if status_code != 200:
    raise ValueError("Failed to log in")
agent = DQNAgent(state_size, action_size, access_token)

# DQN 모델 학습
def train_dqn(episodes=1000):
    batch_size = 32
    total_rewards = []

    for e in range(episodes):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        total_reward = 0

        for time in range(500):
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            total_reward += reward
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print(f"episode: {e}/{episodes}, score: {time}, e: {agent.epsilon:.2}")
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)

        total_rewards.append(total_reward)
        agent.save("dqn_model.h5")
    return total_rewards

# 학습 및 평가 수행
total_rewards = train_dqn(episodes=1000)
average_f1_score = evaluate_recommender(agent, env, users, recipes, interactions)

# 누적 보상 시각화
import matplotlib.pyplot as plt

plt.plot(total_rewards)
plt.title('Total Rewards per Episode')
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.show()